In [ ]:
import pandas as pd
test = pd.read_csv(r"H:\Personal Repos and Projects\Binary Classification with dataset\Binary-Classification-with-a-Bank-Dataset\test.csv")
train = pd.read_csv(r"H:\Personal Repos and Projects\Binary Classification with dataset\Binary-Classification-with-a-Bank-Dataset\train.csv")

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
X = train[train.columns[~train.columns.isin(['y'])]]
X = pd.get_dummies(X)
y = train['y'].to_list()
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, stratify=y)
clf = RandomForestClassifier(max_depth=5, random_state=0)
clf.fit(X, y)

RandomForestClassifier(max_depth=5, random_state=0)

In [25]:
import optuna
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

X = train[train.columns[~train.columns.isin(['y'])]]

X = pd.get_dummies(X)

y = train['y'].to_list()

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, stratify=y)

def objective(trial):
    # Suggest hyperparameters
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 3, 30)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 40)

    # Model
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        random_state=42,
        n_jobs=-1
    )
    
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_valid)[:, 1]

    auc = roc_auc_score(y_valid, preds)
    return auc  


study = optuna.create_study(direction="maximize") 
study.optimize(objective, n_trials=50)

print("Best trial:", study.best_trial.params)
print("Best ROC AUC:", study.best_value)


[I 2025-08-25 19:30:57,405] A new study created in memory with name: no-name-24b8db7c-f210-4ea3-878b-427b821b28ad
[W 2025-08-25 19:31:02,787] Trial 0 failed with parameters: {'n_estimators': 131, 'max_depth': 27, 'min_samples_split': 38} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\znay\AppData\Roaming\Python\Python310\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\znay\AppData\Local\Temp\ipykernel_17316\2486233551.py", line 30, in objective
    model.fit(X_train, y_train)
  File "C:\Users\znay\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "C:\Users\znay\AppData\Roaming\Python\Python310\site-packages\sklearn\ensemble\_forest.py", line 487, in fit
    trees = Parallel(
  File "C:\Users\znay\AppData\Roaming\Python\Python310\site-packages\sklearn\utils\parallel.py"

KeyboardInterrupt: 